<table>
  <tr>
    <td><p style="font-size:45px; color: #55BBD2">Analysis of light microscopy images in Python</p></td>
    <td><img src="../ressources/lmb_logo.svg" alt="LMB Logo" width="500" height="600" align="right"></td>
  </tr>
</table>
<table>
  <tr>
    <td><p style="font-size:15px; color: #55BBD2">Version: September 2025</p></td>
  </tr>
</table>

# Part 3: Particle tracking

<b>Problem:</b> Track transcription factor CDX2

<b>Dataset:</b> 181228_CDX2_9s_c48_n073.tif

<b>Credit:</b> Kuhn, Timo; Hettich, Johannes; Davtyan, Rubina; Gebhardt, J. Christof M. (2021), Single molecule tracking and analysis framework including theory-predicted parameter settings, Scientific Reports, Journal-article, [https://doi.org/10.1038/s41598-021-88802-7](https://doi.org/10.1038/s41598-021-88802-7). [Link to dataset](https://datadryad.org/dataset/doi:10.5061/dryad.0zpc866wh).

<b>Objectives:</b>
- Filter an image (skimage.filters.gaussian, skimage.filters.median)
- Segment an image using a simple thresholding (comparison operators >=, skimage.filters.threshold_otsu)
- Visualize segmentation results (skimage.measure.find_contours)
- Post process binary masks (skimage.morphology.remove_small_objects, skimage.morphology.closing, morphology.footprint_rectangle, skimage.segmentation.clear_border, skimage.color.label2rgb, scipy.ndimage.binary_fill_holes)
- Segment overlapping objects using a watershed-based segmentation (scipy.ndimage.distance_transform_edt, skimage.segmentation.watershed)
- Convert the bit depth of an image (numpy.ndarray.astype)
- Detect spots and generate an image of spot labels from the coordinates (skimage.feature.blob_dog, skimage.util.label_points)
- Measure information within a given ROI (skimage.measure.regionprops_table)
- Use object parenting to count spots per cell (the def keyword)
- Manipulate and structure tabular data (pandas.DataFrame, pandas.merge, pandas.DataFrame.to_csv)

<b>Workflow:</b>

<img src="../ressources/workflow/workflow3.png" alt="drawing" width="800"/>

In [ ]:
from pathlib import Path
from bioio import BioImage

data_folder = Path('../data')
particles = BioImage(Path(data_folder, '181228_CDX2_9s_c48_n073.tif'))
particles_data = particles.data

sz = particles_data.shape
print(sz)

## Napari display
Now we are going to display the particles images, consisting of 105 frames, using the napari plugin. To be able to visualize the 3D rendering of the data, we need to ensure we are displaying a 3D image in one layer. In other words, we need to specify the channel and z = 0 in the viewer i.e display particles_data[:,0,0].

In [ ]:
import napari

v = napari.Viewer()
v.add_image(particles_data[:,0,0])

## Detect each particle (spot) using blob_dog from the feature module

The blob_dog is another function for spot detection. It is a faster approximation of the LoG approach and has few parameters that we need to care for in order to filter out what we consider particles and which ones are not. There are importants parameters that we will cover here: threshold, min_sigma and max_sigma. 

- threshold: The absolute lower bound for scale space maxima. Local maxima smaller than threshold are ignored. Reduce this to detect blobs with lower intensities. <br>
- min_sigma: The minimum standard deviation for Gaussian kernel.
- max_sigma: The maximum standard deviation for Gaussian kernel.

Further reading: https://scikit-image.org/docs/stable/api/skimage.feature.html#skimage.feature.blob_dog

We will save the positions of the detected particles in a panda dataframe.

In [ ]:
from skimage import feature 
import pandas as pd

particles_positions = []

for t in range(sz[0]):    
    coordinates = feature.blob_dog(particles_data[t].squeeze(), min_sigma=1, max_sigma=5, threshold=0.005)    
    particles_positions.append(pd.DataFrame({'x': coordinates[:,1], 'y': coordinates[:,0], 'w': coordinates[:,2], 'frame':t}))

particles_positions = pd.concat(particles_positions)
    
particles_positions

We do not have tracks yet here but points. We can use add_points to add the points to the napari viewer. We will need to select the dataframe from the the TYX information, where T is the timepoints or frames.

<div class="alert alert-success">
       
#### Exercise 

Add the points to the napari viewer using particles_positions[['frame', 'y', 'x']]. Set the size to 1, opacity to 0.5, and face_color to 'green'.
    
</div>

<div class="alert alert-success">

#### Exercise       

The results we have here were optimized using threshold = 0.005, min_sigma = 1, and max_sigma = 5. You may find a better setting than these. <br> 
So go back to the spot detection cell and play with the values of those three parameters and observe how the results are changing accordingly.

   
</div>

## Particles trajectories
We are now going to trace the trajectories of the particles by linking the coordinates positions. <br> 
For this, we will use a Python package named 'laptrack'.

In [ ]:
from laptrack import LapTrack

lt = LapTrack(cutoff=5**2)
track_df, _, _ = lt.predict_dataframe(particles_positions, ["y", "x"], only_coordinate_cols=False)
track_df = track_df.reset_index()

v.add_tracks(track_df[["track_id", "frame", "y", "x"]], tail_length=50)

<div class="alert alert-success">

#### Exercise       

Laptrack has a hands-on list of parameters that we can tune to improve the tracking results. Check them [here](https://laptrack.readthedocs.io/en/stable/reference.html) and play with the parameters to understand how it works.
   
</div>

## Additional constraints

But maybe we eventually want to remove particles that are only appearing in few frames?

Let say we won't consider the trajectories of particles that are appearing in less than 10 frames.

In [ ]:
import pandas as pd

l = track_df['track_id'].unique() 
allowed_track_length = 10
particles_id = [k for k in l if len(track_df.loc[track_df['track_id']==k])>=allowed_track_length]

selected_tracks = pd.DataFrame([])

for t in range(len(particles_id)):
    coordinates = track_df.loc[track_df['track_id']==particles_id[t]]
    selected_tracks = pd.concat([selected_tracks, coordinates])

selected_tracks

<div class="alert alert-success">

#### Exercise       

Change the allowed_track_length and observe the number of trajectories displayed in the overlay display.
   
</div>

## Exporting tabular results

To save results in a csv file use the export function from pandas. <br>
The excelsheet will be save in your current directory. To check where is that directory, run the command pwd in a new cell and you will see the csv file be saved in that directory.


In [ ]:
pwd

In [ ]:
selected_tracks.to_csv('tracking_results.csv')

# Some more helpful tips

- scientific images are data, they can be compromised by inappropriate manipulations
- take good images:
    - don't oversaturate your data
    - use the full dynamic range when taking your images
- do not segment on the data you are measuring, use a housekeeping channel
- images that are compared to each other need to be processed and acquired in the same manner